# Querying with ranges
This is a small tutorial showing how you can accelerate the queries of large tables by using `ctable.whereblocks()` and a combination of boolean conditions and the selections of columns.

In [1]:
import bcolz
import numpy as np

In [2]:
bcolz.print_versions()

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
bcolz version:     1.2.1
NumPy version:     1.16.2
Blosc version:     1.14.3 ($Date:: 2018-04-06 #$)
Blosc compressors: ['blosclz', 'lz4', 'lz4hc', 'snappy', 'zlib', 'zstd']
Numexpr version:   2.6.9
Dask version:      1.2.2
Python version:    3.7.2 (default, Dec 29 2018, 00:00:04) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Platform:          darwin-x86_64
Byte-ordering:     little
Detected cores:    4
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


## Create a table

In [3]:
# The number of entries in the table
N = int(1e7)

In [4]:
# Create the table with two fields, one integer and the other double
ct = bcolz.fromiter(((i,i*i) for i in range(N)), dtype="i4,f8", count=N)
ct

ctable((10000000,), [('f0', '<i4'), ('f1', '<f8')])
  nbytes: 114.44 MB; cbytes: 12.93 MB; ratio: 8.85
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
[(0, 0.) (1, 1.) (2, 4.) ..., (9999997, 9.999994e+13)
 (9999998, 9.999996e+13) (9999999, 9.999998e+13)]

## Add a timestamp

In [5]:
# Create some arbitrary timestamps
import datetime
now = datetime.datetime.timestamp(datetime.datetime.now())
timestamps = bcolz.fromiter((now + 60 * i for i in range(N)), dtype=np.float64, count=N)
timestamps

carray((10000000,), float64)
  nbytes := 76.29 MB; cbytes := 1.39 MB; ratio: 54.72
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
  chunklen := 65536; chunksize: 524288; blocksize: 524288
[  1.55800858e+09   1.55800864e+09   1.55800870e+09 ...,   2.15800840e+09
   2.15800846e+09   2.15800852e+09]

In [6]:
# Add the timestamps top the table, in the field "timestamp" (note that the trailing 's' is out now)
ct.addcol(timestamps, name="timestamp")
ct

ctable((10000000,), [('f0', '<i4'), ('f1', '<f8'), ('timestamp', '<f8')])
  nbytes: 190.73 MB; cbytes: 14.32 MB; ratio: 13.32
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
[(0, 0., 1.55800858e+09) (1, 1., 1.55800864e+09) (2, 4., 1.5580087e+09)
 ..., (9999997, 9.999994e+13, 2.1580084e+09)
 (9999998, 9.999996e+13, 2.15800846e+09)
 (9999999, 9.999998e+13, 2.15800852e+09)]

## Playing with `ctable.whereblocks`

In [7]:
# Do a query with the whole range of timestamps
%time res = [row for row in ct.whereblocks("timestamp > 0", outcols="f0")]

CPU times: user 1.32 s, sys: 44.5 ms, total: 1.36 s
Wall time: 1.41 s


In [8]:
# Do a query in a much more small time range
cond = "(timestamp > now + 100) & (timestamp < now + 1e6)"
%time [row for row in ct.whereblocks(cond)]

CPU times: user 102 ms, sys: 14.9 ms, total: 117 ms
Wall time: 73.6 ms


[array([(    2, 4.00000000e+00, 1.55800870e+09),
        (    3, 9.00000000e+00, 1.55800876e+09),
        (    4, 1.60000000e+01, 1.55800882e+09), ...,
        (16664, 2.77688896e+08, 1.55900842e+09),
        (16665, 2.77722225e+08, 1.55900848e+09),
        (16666, 2.77755556e+08, 1.55900854e+09)],
       dtype=[('f0', '<i4'), ('f1', '<f8'), ('timestamp', '<f8')])]

In [9]:
# Create a boolean array in-memory for ultimate speed
carray_cond = bcolz.eval("(timestamps > now + 100) & (timestamps < now + 1e6)")
carray_cond

carray((10000000,), bool)
  nbytes := 9.54 MB; cbytes := 257.10 KB; ratio: 37.98
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
  chunklen := 262144; chunksize: 262144; blocksize: 65536
[False False  True ..., False False False]

Note that compression ratio is very good, to the point that it usually fits in CPU caches.

In [10]:
# Now, re-run with the boolean array:
%time [row for row in ct.whereblocks(carray_cond, outcols="f0")]

CPU times: user 3.74 ms, sys: 733 µs, total: 4.47 ms
Wall time: 3.56 ms


[array([(    2,), (    3,), (    4,), ..., (16664,), (16665,), (16666,)],
       dtype=[('f0', '<i4')])]

In [11]:
# Get rid of the structured type and use just the dtype for the integer field
%time [row.astype(ct.dtype['f0']) for row in ct.whereblocks(carray_cond, outcols="f0")]

CPU times: user 5.31 ms, sys: 1.18 ms, total: 6.5 ms
Wall time: 5.42 ms


[array([    2,     3,     4, ..., 16664, 16665, 16666], dtype=int32)]

## Apply vectorized operations to the blocks

In [12]:
# Now, perform some bytewise operation on each element
def some_bitwise_op(x):
    y = np.empty(x.shape, x.dtype)
    for i in range(len(x)):
        y[i] = x[i] >> 3 & 0xFF
    return y
%time [some_bitwise_op(row.astype(ct.dtype['f0'])) for row in ct.whereblocks(carray_cond, outcols="f0")]

CPU times: user 58.3 ms, sys: 2.68 ms, total: 61 ms
Wall time: 59.9 ms


[array([ 0,  0,  0, ..., 35, 35, 35], dtype=int32)]

In [13]:
# Now, compile the function with numba
import numba
@numba.jit(nopython=True, cache=True)
def some_bitwise_op_numba(x):
    y = np.empty(x.shape, x.dtype)
    for i in range(len(x)):
        y[i] = x[i] >> 3 & 0xFF
    return y
%time [some_bitwise_op_numba(row.astype(ct.dtype['f0'])) for row in ct.whereblocks(carray_cond, outcols="f0")]

CPU times: user 100 ms, sys: 21.5 ms, total: 122 ms
Wall time: 186 ms


[array([ 0,  0,  0, ..., 35, 35, 35], dtype=int32)]

In [14]:
# Ops, we have got too much time.  Re-run again now that the numba has compiled the function 
%time [some_bitwise_op_numba(row.astype(ct.dtype['f0'])) for row in ct.whereblocks(carray_cond, outcols="f0")]

CPU times: user 4.07 ms, sys: 1.03 ms, total: 5.1 ms
Wall time: 4.05 ms


[array([ 0,  0,  0, ..., 35, 35, 35], dtype=int32)]

Hey, numba is really, really fast.  That's all folks!